# AIG230 NLP (Week 3 Lab) — Notebook 2: Statistical Language Models (Train, Test, Evaluate)

This notebook focuses on **n-gram Statistical Language Models (SLMs)**:
- Train **unigram**, **bigram**, **trigram** models
- Handle **OOV** with `<UNK>`
- Apply **smoothing** (Add-k)
- Evaluate with **cross-entropy** and **perplexity**
- Do **next-word prediction** and simple **text generation**

> Industry framing: even if modern systems use neural LMs, n-gram LMs are still useful for
baselines, constrained domains, and for understanding evaluation.


## 0) Setup


In [1]:

import re
import math
import random
from collections import Counter, defaultdict
from typing import List, Tuple, Dict


## 1) Data: domain text you might see in real systems


We use short texts that resemble:
- release notes
- incident summaries
- operational runbooks
- customer support messaging

In practice, you would load thousands to millions of lines.


In [2]:

corpus = [
    "vpn disconnects frequently after windows update",
    "password reset link expired user cannot login",
    "api requests timeout when latency spikes",
    "portal returns 500 error after deployment",
    "email delivery delayed messages queued",
    "mfa prompt never arrives user stuck at login",
    "wifi drops in meeting rooms access point reboot helps",
    "outlook search not returning results index corrupted",
    "printer driver install fails with error 1603",
    "teams calls choppy audio jitter high",
    "permission denied accessing shared drive though in correct group",
    "battery drains fast after bios update power settings unchanged",
    "push notifications not working on android app",
    "mailbox full cannot receive emails auto archive not running",
]

# Train/test split at sentence level
random.seed(42)
random.shuffle(corpus)
split = int(0.75 * len(corpus))
train_texts = corpus[:split]
test_texts = corpus[split:]

len(train_texts), len(test_texts), train_texts[:2], test_texts[:2]


(10,
 4,
 ['printer driver install fails with error 1603',
  'push notifications not working on android app'],
 ['email delivery delayed messages queued',
  'vpn disconnects frequently after windows update'])

## 2) Tokenization + special tokens


We will:
- lowercase
- keep alphanumerics
- split on whitespace
- add sentence boundary tokens: `<s>` and `</s>`

We will also map rare tokens to `<UNK>` based on training frequency.


In [3]:

def tokenize(text: str) -> List[str]:
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.split()

def add_boundaries(tokens: List[str], n: int) -> List[str]:
    # For n-grams, prepend (n-1) start tokens for simpler context handling
    return ["<s>"]*(n-1) + tokens + ["</s>"]

# Example
tokens = tokenize("Printer driver install fails with error 1603")
add_boundaries(tokens, n=3)


['<s>',
 '<s>',
 'printer',
 'driver',
 'install',
 'fails',
 'with',
 'error',
 '1603',
 '</s>']

## 3) Build vocabulary and handle OOV with <UNK>


In [4]:

# Build vocab from training data
train_tokens_flat = []
for t in train_texts:
    train_tokens_flat.extend(tokenize(t))

freq = Counter(train_tokens_flat)

# Typical practical rule: map tokens with frequency <= 1 to <UNK> in small corpora
min_count = 2
vocab = {w for w, c in freq.items() if c >= min_count}
vocab |= {"<UNK>", "<s>", "</s>"}

def replace_oov(tokens: List[str], vocab: set) -> List[str]:
    return [tok if tok in vocab else "<UNK>" for tok in tokens]

# Show OOV effect
sample = tokenize(test_texts[0])
sample, replace_oov(sample, vocab)


(['email', 'delivery', 'delayed', 'messages', 'queued'],
 ['<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>'])

## 4) Train n-gram counts (unigram, bigram, trigram)


We will compute:
- `ngram_counts[(w1,...,wn)]`
- `context_counts[(w1,...,w_{n-1})]`

Then probability:
\ndefault:  P(w_n | context) = count(context + w_n) / count(context)

This fails when an n-gram is unseen, so we add smoothing.


In [5]:

def get_ngrams(tokens: List[str], n: int) -> List[Tuple[str, ...]]:
    return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

def train_ngram_counts(texts: List[str], n: int, vocab: set) -> Tuple[Counter, Counter]:
    ngram_counts = Counter()
    context_counts = Counter()
    for text in texts:
        toks = replace_oov(tokenize(text), vocab)
        toks = add_boundaries(toks, n)
        for ng in get_ngrams(toks, n):
            ngram_counts[ng] += 1
            context = ng[:-1]
            context_counts[context] += 1
    return ngram_counts, context_counts

uni_counts, uni_ctx = train_ngram_counts(train_texts, n=1, vocab=vocab)
bi_counts, bi_ctx   = train_ngram_counts(train_texts, n=2, vocab=vocab)
tri_counts, tri_ctx = train_ngram_counts(train_texts, n=3, vocab=vocab)

len(uni_counts), len(bi_counts), len(tri_counts)


(5, 10, 17)

## 5) Add-k smoothing and probability function


Add-k smoothing (a common baseline):
\na) Add *k* to every possible next word count  
b) Normalize by context_count + k * |V|

P_k(w|h) = (count(h,w) + k) / (count(h) + k*|V|)

Where V is the vocabulary.


In [6]:

def prob_addk(ngram: Tuple[str, ...], ngram_counts: Counter, context_counts: Counter, V: int, k: float = 0.5) -> float:
    context = ngram[:-1]
    return (ngram_counts[ngram] + k) / (context_counts[context] + k * V)

V = len(vocab)
# Example: P("login" | "<s>") in bigram model
example = ("<s>", "login")
prob_addk(example, bi_counts, bi_ctx, V, k=0.5)


0.038461538461538464

## 6) Evaluate: cross-entropy and perplexity on test set


We evaluate an LM by how well it predicts held-out text.

Cross-entropy (average negative log probability):
H = - (1/N) * sum log2 P(w_i | context)

Perplexity:
PP = 2^H

Lower perplexity is better.


In [7]:

def evaluate_perplexity(texts: List[str], n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, k: float = 0.5) -> float:
    V = len(vocab)
    log2_probs = []
    token_count = 0

    for text in texts:
        toks = replace_oov(tokenize(text), vocab)
        toks = add_boundaries(toks, n)
        ngrams = get_ngrams(toks, n)
        for ng in ngrams:
            p = prob_addk(ng, ngram_counts, context_counts, V, k=k)
            log2_probs.append(math.log(p, 2))
            token_count += 1

    H = -sum(log2_probs) / token_count
    PP = 2 ** H
    return PP

pp_uni = evaluate_perplexity(test_texts, n=1, ngram_counts=uni_counts, context_counts=uni_ctx, vocab=vocab, k=0.5)
pp_bi  = evaluate_perplexity(test_texts, n=2, ngram_counts=bi_counts,  context_counts=bi_ctx,  vocab=vocab, k=0.5)
pp_tri = evaluate_perplexity(test_texts, n=3, ngram_counts=tri_counts, context_counts=tri_ctx, vocab=vocab, k=0.5)

pp_uni, pp_bi, pp_tri


(1.8224739937573902, 1.8712095221558307, 1.9552746520172761)

## 7) Next-word prediction (top-k)


Given a context, compute the probability of each candidate next token and return the top-k.

This mirrors:
- autocomplete in constrained domains
- template suggestion systems
- command prediction in runbooks


In [8]:

def next_word_topk(context_tokens: List[str], n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, k_smooth: float = 0.5, top_k: int = 5):
    # Context length should be n-1
    V = len(vocab)
    context = tuple(context_tokens[-(n-1):]) if n > 1 else tuple()
    candidates = []
    for w in vocab:
        if w in {"<s>"}:
            continue
        ng = context + (w,)
        p = prob_addk(ng, ngram_counts, context_counts, V, k=k_smooth)
        candidates.append((w, p))
    candidates.sort(key=lambda x: -x[1])
    return candidates[:top_k]

# Bigram: context is 1 token
next_word_topk(["<s>"], n=2, ngram_counts=bi_counts, context_counts=bi_ctx, vocab=vocab, top_k=8)


[('<UNK>', 0.8076923076923077),
 ('after', 0.038461538461538464),
 ('error', 0.038461538461538464),
 ('</s>', 0.038461538461538464),
 ('not', 0.038461538461538464)]

## 8) Simple generation (bigram or trigram)


Text generation is not the main goal in SLMs, but it helps you verify:
- boundary handling
- smoothing
- OOV decisions

We will sample tokens until we hit `</s>`.


In [9]:

def sample_next(context_tokens: List[str], n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, k_smooth: float = 0.5):
    V = len(vocab)
    context = tuple(context_tokens[-(n-1):]) if n > 1 else tuple()
    words = [w for w in vocab if w != "<s>"]
    probs = []
    for w in words:
        ng = context + (w,)
        probs.append(prob_addk(ng, ngram_counts, context_counts, V, k=k_smooth))
    # Normalize
    s = sum(probs)
    probs = [p/s for p in probs]
    return random.choices(words, weights=probs, k=1)[0]

def generate(n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, max_len: int = 20, k_smooth: float = 0.5):
    tokens = ["<s>"]*(n-1) if n > 1 else []
    out = []
    for _ in range(max_len):
        w = sample_next(tokens, n, ngram_counts, context_counts, vocab, k_smooth=k_smooth)
        if w == "</s>":
            break
        out.append(w)
        tokens.append(w)
    return " ".join(out)

for _ in range(5):
    print("BIGRAM:", generate(2, bi_counts, bi_ctx, vocab, max_len=18))


BIGRAM: after after <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
BIGRAM: <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> error <UNK> <UNK>
BIGRAM: error after <UNK>
BIGRAM: <UNK> <UNK> <UNK> <UNK> <UNK> not
BIGRAM: <UNK> <UNK> <UNK>


## 9) Model comparison: effect of n and smoothing


Try different `k` values. Notes:
- `k=1.0` is Laplace smoothing (often too strong)
- smaller `k` (like 0.1 to 0.5) is often better

In real corpora, trigrams often beat bigrams, but require more data.


In [10]:

for k in [1.0, 0.5, 0.1, 0.01]:
    pp_bi_k  = evaluate_perplexity(test_texts, n=2, ngram_counts=bi_counts,  context_counts=bi_ctx,  vocab=vocab, k=k)
    pp_tri_k = evaluate_perplexity(test_texts, n=3, ngram_counts=tri_counts, context_counts=tri_ctx, vocab=vocab, k=k)
    print(f"k={k:>4}:  bigram PP={pp_bi_k:,.2f}   trigram PP={pp_tri_k:,.2f}")


k= 1.0:  bigram PP=1.95   trigram PP=2.09
k= 0.5:  bigram PP=1.87   trigram PP=1.96
k= 0.1:  bigram PP=1.79   trigram PP=1.80
k=0.01:  bigram PP=1.76   trigram PP=1.75


## Exercises (do these during lab)
1) Add 20 more realistic domain sentences to the corpus and re-run training/evaluation.  
2) Change `min_count` (OOV threshold) and explain how perplexity changes.  
3) Implement **backoff**: if a trigram is unseen, fall back to bigram; if unseen, fall back to unigram.  
4) Create a function that returns **top-5 next words** given a phrase like: `"user cannot"`.
